In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from joblib import dump
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    auc,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [2]:
df = pd.read_csv("../../../Datasets/SocialNetworkAds.csv")
df.head()

Gender  Age  EstimatedSalary  Purchased
0       1   19            19000          0
1       1   35            20000          0
2       2   26            43000          0
3       2   27            57000          0
4       1   19            76000          0

In [3]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [5]:
estimators = [
    ("rf", RandomForestClassifier(n_estimators=10, random_state=42)),
    ("svr", SVC(probability=True, random_state=42)),
]
stacking = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
stacking.fit(X_train, y_train)
y_pred = stacking.predict(X_test)

In [6]:
stacking_con_matrix = confusion_matrix(y_test, y_pred)
stacking_acc = accuracy_score(y_test, y_pred)
print("Confusion Matrix", "\n", stacking_con_matrix, "\n")
print("Accuracy of Stacking Classification :", stacking_acc * 100, "\n")  # type: ignore
print("Classification Report", "\n", classification_report(y_test, y_pred))

Confusion Matrix 
 [[30  2]
 [ 1  7]] 

Accuracy of Stacking Classification : 92.5 

Classification Report 
               precision    recall  f1-score   support

           0       0.97      0.94      0.95        32
           1       0.78      0.88      0.82         8

    accuracy                           0.93        40
   macro avg       0.87      0.91      0.89        40
weighted avg       0.93      0.93      0.93        40



In [8]:
y_score = stacking.predict_proba(X_test)[:, 1]  # type: ignore
false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, y_score)
roc_auc = auc(false_positive_rate, true_positive_rate)

In [10]:
print(
    "Area Under the Receiver Operating Characteristic Curve :",
    roc_auc_score(y_test, y_score),
)

Area Under the Receiver Operating Characteristic Curve : 0.97265625


In [11]:
dump(stacking, "../../../Models/stackingClassifier.joblib")

['../../../Models/stackingClassifier.joblib']